In [1]:
# Models
from END import EnsembleND
import NestedDichotomies.nd as nd
from sklearn.ensemble import RandomForestClassifier
from PairwiseCoupling import PairwiseCoupling
from sklearn.neural_network import MLPClassifier
# Baselearner
from sklearn import tree
from sklearn import neighbors
from sklearn.linear_model import LogisticRegression
# Methods
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import calibration_curve
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import LabelEncoder
# Basics
import os
from threading import Thread
import openml
import numpy as np
from sklearn.metrics import make_scorer
import warnings;
warnings.filterwarnings('ignore');

In [2]:
def brier_score(y_predict, y_test, nclass):
    obj_num = np.size(y_test)
    bs_ytrue = np.zeros((obj_num,nclass))
    for i in range(obj_num):
        bs_ytrue[i,y_test[i]]=1
    bs = sum(sum((y_predict-bs_ytrue)**2))/obj_num
    return bs

In [3]:
from modelcombos.END_DT import EnsembleND_DT as END_DT
from modelcombos.END_LR import EnsembleND_LR as END_LR
from modelcombos.END_NB import EnsembleND_NB as END_NB
from modelcombos.PC_DT import PairwiseCoupling_DT as PC_DT
from modelcombos.PC_LR import PairwiseCoupling_LR as PC_LR
from modelcombos.PC_NB import PairwiseCoupling_NB as PC_NB
def generateModelName(model):
    name = 'error'
    if (model.__class__==RandomForestClassifier):
        name = 'RF'
    if (model.__class__==PC_DT):
        name = 'PC_DT'
    if (model.__class__==PC_LR):
        name = 'PC_LR'
    if (model.__class__==PC_NB):
        name = 'PC_NB'
    if (model.__class__==MLPClassifier):
        name = 'MLP'
    if (model.__class__==END_DT):
        name = 'END_DT'
    if (model.__class__==END_LR):
        name = 'END_LR'
    if (model.__class__==END_NB):
        name = 'END_NB'
    return name

In [4]:
def brier_score_singular_factory(nclass):
    #def brier_score_singular(y_test, y_predict):
    #    bs_ytrue = np.zeros((nclass))
    #    bs_ytrue[y_test]=1
    #    bs = sum((y_predict-bs_ytrue)**2)
    #    return bs
    def brier_score_singular(y_test,y_predict):
        #print(y_predict)
        #print(y_test)
        obj_num = np.size(y_test)
        bs_ytrue = np.zeros((obj_num,nclass))
        for i in range(obj_num):
            bs_ytrue[i,y_test[i]]=1
        bs = sum(sum((y_predict-bs_ytrue)**2))/obj_num
        return bs
    
    return brier_score_singular

In [5]:
def TMTB_and_ECE(y_predict, y_test, nclass, nbins=10, ccstrat='uniform'):
    y_pred_list = np.reshape(y_predict,nclass*y_test.size)
    ninst = y_test.size
    onehot = np.zeros((ninst, nclass))
    onehot[np.arange(ninst), y_test] = 1
    y_test_list = np.reshape(onehot,nclass*y_test.size)
    prob_true, prob_pred = calibration_curve(y_test_list, y_pred_list, n_bins=nbins, strategy=ccstrat)
    ece = np.sum(np.absolute(prob_true-prob_pred))/prob_true.size
    return ece

In [6]:
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import RandomizedSearchCV
# Compare all models
def CompareModels(data_X, data_y, n_runs=1):
    seed = 2000
    num_classes = np.unique(data_y).size
    # for each run the brierscore for each model [no calibration]
    bs_byModel_base_run = []
    # for each run the ece (expected calibration error) for each model [no calibration]
    ece_byModel_base_run = []
    # for each run the brierscore for each model [sigmoid calibration]
    bs_byModel_sig_run = []
    # for each run the ece (expected calibration error) for each model [sigmoid calibration]
    ece_byModel_sig_run = []
    # for each run the brierscore for each model [isotonic calibration]
    bs_byModel_iso_run = []
    # for each run the ece (expected calibration error) for each model [isotonic calibration]
    ece_byModel_iso_run = []
    # for each run the hyperparameter for each model
    hyperparam_byModel_run = []
    # for each run the mean fit time for each model
    mft_byModel_run = []
    # for each run the mean score time for each model
    mst_byModel_run = []
    # RUNS
    for i in range(n_runs):
        print('run',i+1)
        bs_byModel_base_run.append([])
        ece_byModel_base_run.append([])
        bs_byModel_sig_run.append([])
        ece_byModel_sig_run.append([])
        bs_byModel_iso_run.append([])
        ece_byModel_iso_run.append([])
        hyperparam_byModel_run.append([])
        mft_byModel_run.append([])
        mst_byModel_run.append([])
        # train - test von data (80/20)
        X_train, X_test, y_train, y_test = tts(data_X, data_y, test_size=0.2, stratify=data_y, random_state=seed)
        # model - calibration split von train (70/30)
        X_model, X_calibration, y_model, y_calibration = tts(X_train, y_train, test_size=0.3, stratify=y_train, random_state=seed+1)
        
        scoring = {'bs': make_scorer(brier_score_singular_factory(num_classes), greater_is_better=False, needs_proba=True)}
        
        model_dists = generate_ModelHyperparam_pairs(num_classes, seed)
        
        for k in range(len(model_dists)):
            print('model',k+1)
            #if (k==1):
            #    set_trace()
            model_rs = RandomizedSearchCV(model_dists[k][0], param_distributions=model_dists[k][1], scoring=scoring,refit='bs', n_iter = 10, n_jobs = 10, cv=3, random_state=seed)
            model_rs.fit(X_model, y_model)
            print('mean fit time:',model_rs.cv_results_['mean_fit_time'].mean())
            print('mean score time:',model_rs.cv_results_['mean_score_time'].mean())
            seed += 1
            print(model_rs.best_params_)
            # best estimator
            hyperparam_byModel_run[i].append(model_rs.best_params_)
            model = model_rs.best_estimator_
            #calibration sigmoid
            c_sig_model = CalibratedClassifierCV(base_estimator=model,method='sigmoid', cv='prefit')
            c_sig_model.fit(X_calibration, y_calibration)
            #calibration isotonic
            c_iso_model = CalibratedClassifierCV(base_estimator=model,method='isotonic', cv='prefit')
            c_iso_model.fit(X_calibration, y_calibration)
            #prediction base
            y_pred_base = model.predict_proba(X_test)
            bs_base = brier_score(y_predict=y_pred_base, y_test=y_test, nclass=num_classes)
            ece_base = TMTB_and_ECE(y_predict=y_pred_base, y_test=y_test, nclass=num_classes, nbins=10, ccstrat='uniform')
            #prediction sigmoid calibrated model
            y_pred_sig = c_sig_model.predict_proba(X_test)
            bs_sig = brier_score(y_predict=y_pred_sig, y_test=y_test, nclass=num_classes)
            ece_sig = TMTB_and_ECE(y_predict=y_pred_sig, y_test=y_test, nclass=num_classes, nbins=10, ccstrat='uniform')
            #prediction isotonic calibrated model
            y_pred_iso = c_iso_model.predict_proba(X_test)
            bs_iso = brier_score(y_predict=y_pred_iso, y_test=y_test, nclass=num_classes)
            ece_iso = TMTB_and_ECE(y_predict=y_pred_iso, y_test=y_test, nclass=num_classes, nbins=10, ccstrat='uniform')
            #append results
            bs_byModel_base_run[i].append(bs_base)
            ece_byModel_base_run[i].append(ece_base)
            bs_byModel_sig_run[i].append(bs_sig)
            ece_byModel_sig_run[i].append(ece_sig)
            bs_byModel_iso_run[i].append(bs_iso)
            ece_byModel_iso_run[i].append(ece_iso)
            mft_byModel_run[i].append(model_rs.cv_results_['mean_fit_time'].mean())
            mst_byModel_run[i].append(model_rs.cv_results_['mean_score_time'].mean())
            print(bs_base,ece_base,bs_sig, ece_sig, bs_iso, ece_iso)
    return bs_byModel_base_run, ece_byModel_base_run, bs_byModel_sig_run, ece_byModel_sig_run, bs_byModel_iso_run, ece_byModel_iso_run, hyperparam_byModel_run, mft_byModel_run, mst_byModel_run

In [7]:
from modelcombos.END_DT import EnsembleND_DT as END_DT
from modelcombos.END_LR import EnsembleND_LR as END_LR
from modelcombos.END_NB import EnsembleND_NB as END_NB
from modelcombos.PC_DT import PairwiseCoupling_DT as PC_DT
from modelcombos.PC_LR import PairwiseCoupling_LR as PC_LR
from modelcombos.PC_NB import PairwiseCoupling_NB as PC_NB
from scipy.stats import randint
from scipy.stats import uniform
def generate_ModelHyperparam_pairs(nclasses, seed):
    models = []
    RFC_paramdist =  {
        'min_impurity_decrease': uniform(0.00001, 0.1),
        'min_samples_leaf': randint(1,51)}
    models.append((RandomForestClassifier(n_estimators=45, random_state=seed), RFC_paramdist))
    DT_paramdist = {
        'max_depth': randint(1,4)
    }
    LR_paramdist = {
        'penalty': ['l1', 'l2'],
        'C': uniform(0.01,20)
    }
    NB_paramdist = {
        'var_smoothing' : uniform(0.0000000001,1.0)
    }
    models.append((END_DT(number_of_nds=5, number_of_classes=nclasses, max_depth = 1, generator_String='random_pair', random_state=seed), DT_paramdist))
    models.append((END_LR(number_of_nds=5, number_of_classes=nclasses,penalty='l2', C=1.0, generator_String='random_pair', random_state=seed), LR_paramdist))
    models.append((END_NB(number_of_nds=5, number_of_classes=nclasses, var_smoothing=0.00001, generator_String='random_pair', random_state=seed),NB_paramdist))
    models.append((PC_DT(classes=nclasses, seed=seed, max_depth=1),DT_paramdist))
    models.append((PC_LR(classes=nclasses, seed=seed, penalty='l2', C=1.0),LR_paramdist))
    models.append((PC_NB(classes=nclasses, seed=seed, var_smoothing=0.0001),NB_paramdist))
    MLP_paramdist = {
        'alpha': uniform(0.00000001, 10),
        'batch_size': [100,200,300,400,500],
        'power_t': uniform(0.001,5) 
    }
    models.append((MLPClassifier(hidden_layer_sizes = (35,), activation='logistic', learning_rate='invscaling'), MLP_paramdist))
    return models

In [8]:
from time import gmtime, strftime
    
def write_single_value_per_run(file, models, value_name, values):
    file.write(value_name+'\n')
    file.write(generateModelName(models[0][0]))
    for i in range(len(models)-1):
        file.write('\t'+generateModelName(models[i+1][0]))
    file.write('\n')
    for i in range(len(values)):
        file.write(str(values[i][0]))
        for k in range(len(values[i])-1):
            file.write('\t'+str(values[i][k+1]))
        file.write('\n')
    
def save_result(dataset_name, nclasses, bs_byModel_base_run, ece_byModel_base_run, bs_byModel_sig_run, ece_byModel_sig_run, bs_byModel_iso_run, ece_byModel_iso_run, hyperparam_byModel_run, mft_byModel_run, mst_byModel_run):
    dir_path = os.getcwd()
    directory = dir_path+'/experiments/'+dataset_name+'/'
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    models = generate_ModelHyperparam_pairs(nclasses=nclasses, seed=42)
    file = open(directory+'result_'+strftime("%Y-%m-%d %H_%M_%S", gmtime())+'.txt', 'w')
    # WRITE MODEL DESCRIPTIONS
    try:
        # SAVING BRIER SCORE
        file.write('Brier-Score\n')
        file.write(generateModelName(models[0][0]))
        for i in range(len(models)-1):
            file.write('\t\t\t'+generateModelName(models[i+1][0]))
        file.write('\nBase\tSigmoid\tIsotonic')
        for i in range(len(models)-1):
            file.write('\tBase\tSigmoid\tIsotonic')
        file.write('\n')
        for i in range(len(bs_byModel_base_run)):
            file.write(str(bs_byModel_base_run[i][0])+'\t'+str(bs_byModel_sig_run[i][0])+'\t'+str(bs_byModel_iso_run[i][0]))
            for k in range(len(bs_byModel_base_run[i])-1):
                file.write('\t'+str(bs_byModel_base_run[i][k+1])+'\t'+str(bs_byModel_sig_run[i][k+1])+'\t'+str(bs_byModel_iso_run[i][k+1]))
            file.write('\n')
        # SAVING ECE SCORE
        file.write('ECE-Score\n')
        file.write(generateModelName(models[0][0]))
        for i in range(len(models)-1):
            file.write('\t\t\t'+generateModelName(models[i+1][0]))
        file.write('\nBase\tSigmoid\tIsotonic')
        for i in range(len(models)-1):
            file.write('\tBase\tSigmoid\tIsotonic')
        file.write('\n')
        for i in range(len(ece_byModel_base_run)):
            file.write(str(ece_byModel_base_run[i][0])+'\t'+str(ece_byModel_sig_run[i][0])+'\t'+str(ece_byModel_iso_run[i][0]))
            for k in range(len(ece_byModel_base_run[i])-1):
                file.write('\t'+str(ece_byModel_base_run[i][k+1])+'\t'+str(ece_byModel_sig_run[i][k+1])+'\t'+str(ece_byModel_iso_run[i][k+1]))
            file.write('\n')
        # SAVING HYPERPARAMETER
        write_single_value_per_run(file, models, 'Hyperparameter', hyperparam_byModel_run)
        # SAVING MEAN FIT TIME
        write_single_value_per_run(file, models, 'Mean Fit Time', mft_byModel_run)
        # SAVING MEAN SCORE TIME
        write_single_value_per_run(file, models, 'Mean Score Time', mst_byModel_run)
    finally:
        file.close()

In [9]:
#1515 - micro-mass
#1459 - artificial-characters
#1233 - eating
#1569 - poker-hand
#1503 - spoken-arabic-digit
#4541 - Diabetes130US
#4538 - GesturePhaseSegmentationProcessed
#41991 - Kuzushiji-49 - error
#40670 - dna
#1478 - har
#40984 - segment
#40498 - wine-quality-white
#40499 - texture
#40686 - solar-flare
#41972 - Indian_pines
#1475 - first-order-theorem-proving
dataset = openml.datasets.get_dataset(1459)
dataset_name = dataset.name
print(dataset_name)
X, y, categorical_indicator, attribute_names = dataset.get_data(dataset_format='array', target=dataset.default_target_attribute)
num_classes = np.unique(y).size
#bs & ece for (b)ase, (s)igmoid and (i)sotonic
bs_b, ece_b, bs_s, ece_s, bs_i, ece_i, hyppar, mft, mst = CompareModels(X,y, 1)
save_result(dataset_name=dataset_name, nclasses=num_classes, bs_byModel_base_run=bs_b, ece_byModel_base_run=ece_b, bs_byModel_sig_run=bs_s, ece_byModel_sig_run=ece_s, bs_byModel_iso_run=bs_i, ece_byModel_iso_run=ece_i, hyperparam_byModel_run=hyppar, mft_byModel_run=mft, mst_byModel_run=mst)

artificial-characters
run 1
model 1
mean fit time: 0.09446362654368082
mean score time: 0.03242847919464111
{'min_impurity_decrease': 0.006820628846711264, 'min_samples_leaf': 28}
0.7095073617797227 0.25811497790074217 0.6242072667790545 0.10497712751846511 0.5952020868208618 0.0803100773111178
model 2
mean fit time: 0.35942945480346683
mean score time: 0.028005568186442058
{'max_depth': 3}
0.6827938327880142 0.10365108940146778 0.6688646806144374 0.05800969948473712 0.6458725559069057 0.04825468928843779
model 3
mean fit time: 2.30243178208669
mean score time: 0.016038695971171062
{'C': 1.989002315967564, 'penalty': 'l1'}
0.7503205215511878 0.02168301256888665 0.7565785313345491 0.054110694531687764 0.7323179960485989 0.13685765558655918
model 4
mean fit time: 0.3946847677230835
mean score time: 0.060223897298177086
{'var_smoothing': 0.18836204849738158}
0.8044692683539403 0.04989852167859649 0.7916972685537925 0.058089575705687034 0.772993405367868 0.060599620660182124
model 5
mean f